In [44]:
import baostock as bs
import pandas as pd

# 登陆系统
lg = bs.login()
# 显示登陆返回信息
print('login respond error_code:'+lg.error_code)
print('login respond  error_msg:'+lg.error_msg)

# 获取中证500成分股
rs = bs.query_zz500_stocks()
print('query_zz500 error_code:'+rs.error_code)
print('query_zz500  error_msg:'+rs.error_msg)

# 打印结果集
zz500_stocks = []
while (rs.error_code == '0') & rs.next():
    # 获取一条记录，将记录合并在一起
    zz500_stocks.append(rs.get_row_data())
zz500 = pd.DataFrame(zz500_stocks, columns=rs.fields)# 登出系统
bs.logout()


login success!
login respond error_code:0
login respond  error_msg:success
query_zz500 error_code:0
query_zz500  error_msg:success
logout success!


In [45]:
zz500

,updateDate,code,code_name
0,2022-08-01,sh.600006,东风汽车
1,2022-08-01,sh.600008,首创环保
2,2022-08-01,sh.600021,上海电力
3,2022-08-01,sh.600022,山东钢铁
4,2022-08-01,sh.600026,中远海能
...,...,...,...
495,2022-08-01,sz.300741,华宝股份
496,2022-08-01,sz.300769,德方纳米
497,2022-08-01,sz.300832,新产业
498,2022-08-01,sz.300869,康泰医学


In [ ]:
import datetime
import os.path
import sys
import pickle
import backtrader as bt
from backtrader.indicators import EMA

In [46]:
lg = bs.login()
# 显示登陆返回信息
print('login respond error_code:'+lg.error_code)
print('login respond  error_msg:'+lg.error_msg)

#### 获取沪深A股历史K线数据 ####
# 详细指标参数，参见“历史行情指标参数”章节；“分钟线”参数与“日线”参数不同。“分钟线”不包含指数。
# 分钟线指标：date,time,code,open,high,low,close,volume,amount,adjustflag
# 周月线指标：date,code,open,high,low,close,volume,amount,adjustflag,turn,pctChg
for code in list(zz500.code):
    rs = bs.query_history_k_data_plus(code,
        "date,code,open,high,low,close,preclose,volume,amount,adjustflag,turn,tradestatus,pctChg,isST",
        start_date='2020-01-01', end_date='2023-02-15',
        frequency="d", adjustflag="3")
    data_list = []
    while (rs.error_code == '0') & rs.next():
        # 获取一条记录，将记录合并在一起
        data_list.append(rs.get_row_data())
    result = pd.DataFrame(data_list, columns=rs.fields)

    #### 结果集输出到csv文件 ####   
    result.to_csv(f"stocks/{code}_data.csv", index=False)
    print(code)
    
#### 登出系统 ####
bs.logout()

login success!
login respond error_code:0
login respond  error_msg:success
sh.600006
sh.600008
sh.600021
sh.600022
sh.600026
sh.600027
sh.600037
sh.600038
sh.600039
sh.600056
sh.600060
sh.600062
sh.600064
sh.600066


KeyboardInterrupt: 

In [50]:

class TestStrategy(bt.Strategy):
    """
    构建自己的策略
    """

    def log(self, txt, dt=None, doprint=False):
        ''' 日志函数，用于统一输出日志格式 '''
        if doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):

        # 初始化相关数据
        self.dataclose = self.datas[0].close
        # 成交量
        self.volume = self.datas[0].volume
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # 计算两个EMA均线和MACD信号
        ema1 = EMA(self.data, period=13)
        ema2 = EMA(self.data, period=26)
        self.macd = ema1 - ema2
        self.signal = EMA(self.macd, period=9)
        #画图
        bt.indicators.MACDHisto(self.data)

    def notify_order(self, order):
        """
        订单状态处理
        Arguments:
            order {object} -- 订单状态
        """
        if order.status in [order.Submitted, order.Accepted]:
            # 如订单已被处理，则不用做任何事情
            return

        # 检查订单是否完成
        if order.status in [order.Completed]:
            if order.isbuy():
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            self.bar_executed = len(self)

        # 订单因为缺少资金之类的原因被拒绝执行
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('订单取消或被拒绝',doprint=True)

        # 订单状态处理完成，设为空
        self.order = None

    def notify_trade(self, trade):
        """
        交易成果
        
        Arguments:
            trade {object} -- 交易状态
        """
        if not trade.isclosed:
            return

        # 显示交易的毛利率和净利润
#         self.log('交易中，毛利%.2f, 净利 %.2f' %
#                  (trade.pnl, trade.pnlcomm), doprint=True)

    def next(self):
        ''' 下一次执行 '''

        # 记录收盘价
#         self.log('Close, %.2f' % self.dataclose[0])

        # 是否正在下单，如果是的话不能提交第二次订单
        if self.order:
            return

        # 是否已经买入
        if not self.position:
            #第一个条件，前一天di
            condition1 = self.macd[-1] - self.signal[-1]
            condition2 = self.macd[0] - self.signal[0]
            if condition1 < 0 and condition2 > 0:
#                 self.log('买入, %.2f' % self.dataclose[0],doprint=True)
                self.order = self.buy()
        else:
            condition1 = self.macd[-1] - self.signal[-1]
            condition2 = self.macd[0] - self.signal[0]
            if condition1 > 0 and condition2 < 0:
#                 self.log('卖出, %.2f' % self.dataclose[0],doprint=True)
                self.order = self.sell()

    def stop(self):
        self.log(u'最终的账户资产为%.2f' %
                 (self.broker.getvalue()), doprint=True)

def run_backtest(file_name,result):
    # 初始化模型
    cerebro = bt.Cerebro()

    # 构建策略
    strats = cerebro.addstrategy(TestStrategy)
    # 每次买100股
    cerebro.addsizer(bt.sizers.FixedSize, stake=100)

    # 加载数据到模型中
    data = bt.feeds.GenericCSVData(
        dataname=file_name,
        fromdate=datetime.datetime(2020, 1, 1),
        todate=datetime.datetime(2023, 2, 12),
        dtformat='%Y-%m-%d',
        datetime=0,
        open=2,
        high=3,
        low=4,
        close=5,
        volume=7
    )
    cerebro.adddata(data)

    # 设定初始资金和佣金
    cerebro.broker.setcash(10000.0)
    cerebro.broker.setcommission(0.005)

    # 策略执行前的资金
    print('启动资金: %.2f' % cerebro.broker.getvalue())
    # 策略执行
    cerebro.run()
    # 剩余本金
    money_left = cerebro.broker.getvalue()

    # 获取股票名字
    stock_name = file_name.split('_')[-2]

    # 将最终回报率以百分比的形式返回
    result[stock_name] = money_left

In [51]:
result = {}

# 遍历所有股票数据
for path in os.listdir('stocks\\'):
    print(path)
    run_backtest('stocks/' + path, result)



sh.600006_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9730.78
sh.600008_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9910.87
sh.600021_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9957.18
sh.600022_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为10003.31
sh.600026_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9487.52
sh.600027_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为10104.82
sh.600037_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9839.25
sh.600038_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为10537.44
sh.600039_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为10218.80
sh.600056_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为10271.78
sh.600060_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为10374.86
sh.600062_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为10964.33
sh.600064_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9889.80
sh.600066_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为8555.24
sh.600079_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为8581.86
sh.600095_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9156.14
s

2023-02-10, 最终的账户资产为9999.79
sh.600820_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9883.29
sh.600823_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9949.06
sh.600827_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为11253.75
sh.600835_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9429.34
sh.600839_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9985.39
sh.600848_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为8227.49
sh.600859_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为11822.60
sh.600862_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为10674.58
sh.600863_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9992.02
sh.600867_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9517.40
sh.600871_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9964.71
sh.600875_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为8923.06
sh.600879_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9717.78
sh.600885_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为4423.55
sh.600895_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为8903.78
sh.600901_data.csv
启动资金: 10000.00
2023-

2020-04-10, 订单取消或被拒绝
2020-05-14, 订单取消或被拒绝
2023-02-10, 最终的账户资产为15052.94
sh.603638_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为8693.70
sh.603650_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为11115.22
sh.603658_data.csv
启动资金: 10000.00
2020-03-06, 订单取消或被拒绝
2020-04-02, 订单取消或被拒绝
2020-05-20, 订单取消或被拒绝
2020-07-27, 订单取消或被拒绝
2020-08-28, 订单取消或被拒绝
2020-09-18, 订单取消或被拒绝
2020-09-23, 订单取消或被拒绝
2020-10-29, 订单取消或被拒绝
2020-12-09, 订单取消或被拒绝
2020-12-29, 订单取消或被拒绝
2021-01-08, 订单取消或被拒绝
2021-03-23, 订单取消或被拒绝
2021-05-18, 订单取消或被拒绝
2023-02-03, 订单取消或被拒绝
2023-02-07, 订单取消或被拒绝
2023-02-10, 最终的账户资产为6795.35
sh.603707_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为6478.61
sh.603712_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为8875.20
sh.603719_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为4468.99
sh.603786_data.csv
启动资金: 10000.00
2022-01-17, 订单取消或被拒绝
2023-02-10, 最终的账户资产为7090.06
sh.603858_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9212.56
sh.603866_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为6838.27
sh.603868_data.csv
启动资金: 10000.00


2023-02-10, 最终的账户资产为9732.26
sz.000690_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为10122.11
sz.000709_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为10016.55
sz.000717_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为10067.28
sz.000718_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为10122.17
sz.000728_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9709.90
sz.000729_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9916.36
sz.000738_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为8774.14
sz.000739_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为10367.60
sz.000750_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9902.59
sz.000778_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9959.65
sz.000783_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为10117.44
sz.000785_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9546.66
sz.000807_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为11492.52
sz.000825_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为10076.53
sz.000830_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为10571.52
sz.000831_data.csv
启动资金: 10000.00

2023-02-10, 最终的账户资产为5958.76
sz.002701_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9540.98
sz.002705_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9976.36
sz.002739_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9184.10
sz.002745_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9713.59
sz.002756_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为14994.49
sz.002797_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9948.08
sz.002815_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9036.18
sz.002831_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为10205.38
sz.002867_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为8999.48
sz.002901_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为12549.67
sz.002925_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为6844.77
sz.002926_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9818.67
sz.002936_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9745.35
sz.002939_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为10131.41
sz.002945_data.csv
启动资金: 10000.00
2023-02-10, 最终的账户资产为9329.52
sz.002946_data.csv
启动资金: 10000.00
2023

In [52]:
result

{'stocks/sh.600006': 9730.78,
 'stocks/sh.600008': 9910.87,
 'stocks/sh.600021': 9957.180000000002,
 'stocks/sh.600022': 10003.305,
 'stocks/sh.600026': 9487.519999999997,
 'stocks/sh.600027': 10104.815,
 'stocks/sh.600037': 9839.245,
 'stocks/sh.600038': 10537.444999999998,
 'stocks/sh.600039': 10218.795000000002,
 'stocks/sh.600056': 10271.784999999994,
 'stocks/sh.600060': 10374.859999999993,
 'stocks/sh.600062': 10964.325,
 'stocks/sh.600064': 9889.795,
 'stocks/sh.600066': 8555.235000000004,
 'stocks/sh.600079': 8581.86499999999,
 'stocks/sh.600095': 9156.140000000001,
 'stocks/sh.600096': 10498.214999999993,
 'stocks/sh.600109': 9620.684999999996,
 'stocks/sh.600118': 6694.535000000002,
 'stocks/sh.600120': 9706.675000000001,
 'stocks/sh.600126': 9903.775,
 'stocks/sh.600131': 8980.06499999999,
 'stocks/sh.600141': 9610.34,
 'stocks/sh.600143': 8853.275000000005,
 'stocks/sh.600153': 9439.7,
 'stocks/sh.600155': 9457.929999999998,
 'stocks/sh.600157': 9994.96,
 'stocks/sh.600160'

In [53]:
sum(result.values())

4794311.294999999